In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 38.8 MB/s 
     |████████████████████████████████| 498.0 MB 11 kB/s 
     |████████████████████████████████| 5.8 MB 60.5 MB/s 
     |████████████████████████████████| 1.4 MB 65.8 MB/s 
     |████████████████████████████████| 462 kB 77.7 MB/s 


In [3]:
!pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 38.2 MB/s 
     |████████████████████████████████| 238 kB 76.1 MB/s 
     |████████████████████████████████| 352 kB 69.5 MB/s 
     |████████████████████████████████| 1.1 MB 59.1 MB/s 
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 118 kB 15.3 MB/s 
     |████████████████████████████████| 1.3 MB 10.2 MB/s 


In [4]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

from keras import backend as K 
from natsort import natsorted

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [5]:
import pandas as pd

train_all = pd.read_csv('drive/MyDrive/ML/texts_with_skills_one-hot.csv', sep = "|", encoding = "UTF-8")

In [6]:
#train_all.head()

In [7]:
train_all_X = train_all['text']

In [8]:
#bert_preprocess_model = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_preprocess/1')

In [9]:
#text_preprocessed = bert_preprocess_model(train_all_X)

In [10]:
#bert_model = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_L-12_H-768_A-12/1')

In [11]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_preprocess/1', name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_L-12_H-768_A-12/1', trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [12]:
#classifier_model = build_classifier_model()

In [13]:
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#metrics = tf.metrics.BinaryAccuracy()

In [14]:
#epochs = 10
#steps_per_epoch = 64
#num_train_steps = steps_per_epoch * epochs
#num_warmup_steps = int(0.1*num_train_steps)

#init_lr = 3e-5
#optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                          num_train_steps=num_train_steps,
#                                          num_warmup_steps=num_warmup_steps,
#                                          optimizer_type='adamw')

In [15]:
#classifier_model.compile(optimizer=optimizer,
#                         loss=loss,
#                         metrics=metrics)

In [16]:
#history = classifier_model.fit(x = train_all_X, y = train_0_y, epochs = epochs)

Test

In [17]:
test_all = pd.read_csv('drive/MyDrive/ML/test_texts_with_ids.csv', sep = "|", encoding = "UTF-8")

In [18]:
test_texts = test_all["text"]

In [19]:
#for i in tf.sigmoid(classifier_model(tf.constant(test_texts))):
#    print(i)

In [20]:
to_drop = []

directory = 'drive/MyDrive/ML/preds'

for filename in natsorted(os.listdir(directory)):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        skill = f.strip().split(".")[0].split("_")[-1]
        to_drop.append(skill)

skill_list = train_all.keys().drop('text').drop(to_drop).tolist()

In [21]:
skills_in_test = []

for skill in skill_list:
    K.clear_session()
    tf.keras.backend.clear_session()
    
    print("Class =", skill)
    classifier_model = build_classifier_model()
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()

    epochs = 5
    steps_per_epoch = 64
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')
    
    classifier_model.compile(optimizer=optimizer,
                            loss=loss,
                            metrics=metrics)
    
    train_y = train_all[skill]

    total = len(train_y)
    pos = train_y.value_counts()[1]
    neg = train_y.value_counts()[0]

    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0: weight_for_0, 1: weight_for_1}

    history = classifier_model.fit(x = train_all_X, y = train_y, epochs = epochs, class_weight = class_weight)

    filename = "drive/MyDrive/ML/preds/test_preds_" + skill + ".txt"
    with open(filename, 'w') as fp:
        for item in tf.sigmoid(classifier_model(tf.constant(test_texts))).numpy().tolist():
            fp.write("%s\n" % item[0])
        print('Done')

    del classifier_model
    K.clear_session()

Class = s8
Epoch 1/5
7/7 [==============================] - 23s 925ms/step - loss: 0.6900 - binary_accuracy: 0.9163
Epoch 2/5
7/7 [==============================] - 7s 990ms/step - loss: 0.6784 - binary_accuracy: 0.9163
Epoch 3/5
7/7 [==============================] - 7s 939ms/step - loss: 0.6987 - binary_accuracy: 0.9163
Epoch 4/5
7/7 [==============================] - 7s 971ms/step - loss: 0.6986 - binary_accuracy: 0.9163
Epoch 5/5
7/7 [==============================] - 7s 982ms/step - loss: 0.7097 - binary_accuracy: 0.9163
Done
Class = s9
Epoch 1/5
7/7 [==============================] - 21s 945ms/step - loss: 0.7139 - binary_accuracy: 0.9458
Epoch 2/5
7/7 [==============================] - 7s 961ms/step - loss: 0.6830 - binary_accuracy: 0.9458
Epoch 3/5
7/7 [==============================] - 7s 933ms/step - loss: 0.7234 - binary_accuracy: 0.9458
Epoch 4/5
7/7 [==============================] - 7s 967ms/step - loss: 0.6600 - binary_accuracy: 0.9458
Epoch 5/5
7/7 [====================

In [22]:
import os
from natsort import natsorted
# assign directory
directory = 'drive/MyDrive/ML/preds'

skills_preds = pd.DataFrame()

# iterate over files in
# that directory
for filename in natsorted(os.listdir(directory)):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        skill = f.strip().split(".")[0].split("_")[-1]
        with open(f) as file:
          df = file.readlines()
          skills_preds[skill] = [float(i) for i in df]

In [23]:
truths = pd.concat([test_all.drop('text', axis = 1), skills_preds >= 0.5], axis = 1)

In [24]:
thingie = []

for index, row in truths.iterrows():
  id = row['text_id']

  skl = row.drop('text_id')

  true_skills = skl[skl == True]
  
  if len(true_skills) == 0:
    highest_idx = skills_preds.iloc[index].idxmax()
    true_skills = [highest_idx]
  else:
    true_skills = true_skills.keys().to_list()

  skill_list = " ".join(true_skills)
  thingie.append(",".join([id, skill_list]))

In [25]:
with open("drive/MyDrive/ML/XLM_RoBERTa_5e_withBest.csv", "w") as file:
  file.write("text_id,skills\n")
  for line in thingie:
    file.write(line)
    file.write("\n")